<a href="https://colab.research.google.com/github/amirezzati/Arabic-Broken-Plural-RNN/blob/main/Arabic_broken_plurals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries & Constants**

In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Softmax, BatchNormalization, Dropout, Conv2D, MaxPool2D, Embedding, Reshape, Flatten, LSTM, GRU, SimpleRNN, Concatenate, concatenate, RepeatVector, TimeDistributed, Bidirectional
# from keras.layers.merge import concatenate
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax, Nadam, SGD
from tensorflow.keras import Model
from tensorflow.keras.optimizers.schedules import InverseTimeDecay, ExponentialDecay
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences #from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import os
import string
from google.colab import drive
import json

from transformers import AutoTokenizer,BertTokenizer,TFBertModel

In [ ]:
drive.mount('/content/drive')
dataPath="/content/drive/MyDrive/AI Projects/Arabic Broken Plural/data"
# dataPath = '/content/drive/MyDrive/Deep Learning/Datasets/Arabic Broken Plural/data'

# **Preprocessing**

## **Encoding Schemes**

In [4]:
encoding_schemes = pd.read_excel(dataPath+"/Encoding Schemes.xlsx")

In [5]:
arabic_bw = encoding_schemes[['Arabic', 'BW']]
arabic_bw.shape[0]

51

In [6]:
bw_to_arabic, arabic_to_bw = {}, {}
for i in range(arabic_bw.shape[0]):
  bw_to_arabic[arabic_bw['BW'][i]] = arabic_bw['Arabic'][i]
  arabic_to_bw[arabic_bw['Arabic'][i]] = arabic_bw['BW'][i]

In [7]:
def getArabicForm(bwForm):
  arabicForm = []
  for x in bwForm:
    arabicForm.append("".join([bw_to_arabic[char] for char in x]))
  return arabicForm

## **Hugging face Arabic Bert Config**

In [ ]:
#bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
bert_tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")

In [9]:
encoding = bert_tokenizer.encode_plus(
      'علم',
      max_length=18,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=True,
      padding = 'max_length',  # Pad to longest in batch.
      truncation=True,
      return_attention_mask=True,
      return_tensors='tf'  # Return PyTorch tensors
    )
#print(len(encoding['token_type_ids'][0]))

In [10]:
bert_test=bert_tokenizer("عِلمة")

In [46]:
print(bert_test)

{'input_ids': [2, 2816, 1012, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


## **Get dataset files and Organizing**

In [12]:
train_csv = pd.read_csv(dataPath+"/train_pairs_all.csv")
dev_csv = pd.read_csv(dataPath+"/dev_pairs_all.csv")
test_csv = pd.read_csv(dataPath+"/test_pairs_all.csv")

train_csv.head()

,lemma,inflection,NUM,GEN,RAT,lex_freq,B/S,SING_PATT,PL_PATT,ROOT,FREQ
0,Eamal,>aEomAl,PL,m,i,49,B,1a2a3,>a1o2A3,E.m.l,49
1,hadaf,>ahodAf,PL,m,i,42,B,1a2a3,>a1o2A3,h.d.f,42
2,EuDow,>aEoDA',PL,m,r,46,B,1u2ow,>a1o2A',E.D.#,39
3,jihAz,>ajohizap,PL,m,i,41,B,1i2A3,>a1o2i3ap,j.h.z,35
4,Hukom,>aHokAm,PL,m,i,35,B,1u2o3,>a1o2A3,H.k.m,35


In [ ]:
train_csv

In [14]:
train_csv_filtered = train_csv[train_csv['ROOT'] != 'NTWS']
dev_csv_filtered = dev_csv[dev_csv['ROOT'] != 'NTWS']
test_csv_filtered = test_csv[test_csv['ROOT'] != 'NTWS']

In [15]:
print(train_csv.shape[0] - train_csv_filtered.shape[0])
print(dev_csv.shape[0] - dev_csv_filtered.shape[0])
print(test_csv.shape[0] - test_csv_filtered.shape[0])

114
38
27


In [16]:

# Label encoder for encoding gender and rational features
# 'B' class: 0      'S' class: 1
categorical_attr = ['GEN', 'RAT', 'B/S']

le = LabelEncoder()
#train_csv_filtered[categorical_attr] = train_csv_filtered[categorical_attr].apply(le.fit_transform, axis=0)
train_csv[categorical_attr] = train_csv[categorical_attr].apply(le.fit_transform, axis=0)

le = LabelEncoder()
#dev_csv_filtered[categorical_attr] = dev_csv_filtered[categorical_attr].apply(le.fit_transform, axis=0)
dev_csv[categorical_attr] = dev_csv[categorical_attr].apply(le.fit_transform, axis=0)

le = LabelEncoder()
#test_csv_filtered[categorical_attr] = test_csv_filtered[categorical_attr].apply(le.fit_transform, axis=0)
test_csv[categorical_attr] = test_csv[categorical_attr].apply(le.fit_transform, axis=0)

#test_csv_filtered.head()
test_csv.head()

,lemma,inflection,NUM,GEN,RAT,lex_freq,B/S,SING_PATT,PL_PATT,ROOT,FREQ
0,Eamal,>aEomAl,PL,1,0,23,0,1a2a3,>a1o2A3,E.m.l,23
1,hadaf,>ahodAf,PL,1,0,16,0,1a2a3,>a1o2A3,h.d.f,16
2,EalAqap,EalAqAt,PL,0,0,15,1,1a2A3ap,1a2A3At,E.l.q,15
3,mu&as~asap,mu&as~asAt,PL,0,0,14,1,mu&a2~a3ap,mu&a2~a3At,#.s.s,14
4,EuDow,>aEoDA',PL,1,1,15,0,1u2ow,>a1o2A',E.D.#,13


In [17]:
def getListOfData(dataFrame):
  '''
  Output type->List : 0:lamma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational | 8:freq | 9:nmberOfRoot | 10:Arabic Form |
  '''
  data = [] # 0:lemma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational 
  data.append(dataFrame['lemma'].tolist())
  data.append(dataFrame['SING_PATT'].tolist())

  root = dataFrame['ROOT'].tolist()
  # delete dot sign(.) in root feature
  for i in range(len(root)): 
    root[i] = root[i].replace(".","")
    root[i] = root[i].replace("#","")
  data.append(root)

  data.append(dataFrame['inflection'].tolist())
  data.append(dataFrame['PL_PATT'].tolist())
  data.append(dataFrame['B/S'].tolist())
  data.append(dataFrame['GEN'].tolist())
  data.append(dataFrame['RAT'].tolist())
  data.append(dataFrame['FREQ'].tolist())

  numOfRoot = [len(x) for x in root]
  data.append(numOfRoot)

  data.append(getArabicForm(dataFrame['lemma'].tolist()))

  return data

trainData = getListOfData(train_csv)
devData = getListOfData(dev_csv)
testData = getListOfData(test_csv)
is_dataOrginal=True

In [ ]:
trainData[1]

## **Sentiment Analysis Model**

In [ ]:
pip install camel-tools

In [ ]:
from camel_tools.sentiment import SentimentAnalyzer

sa = SentimentAnalyzer("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")

In [47]:
print(trainData[0][1])
sentences = [trainData[10][0], trainData[0][1]]

sa.predict(sentences)

hadaf


['neutral', 'neutral']

## **Analyzing Datasets**

In [21]:
# generate samples of dev/test set which are not in train set
# compare with lemma
devSetExistence = np.array([lemma in trainData[0] for lemma in devData[0]])
testSetExistence = np.array([lemma in trainData[0] for lemma in testData[0]])

unseenLemmaIndices_dev = np.where(devSetExistence==False)[0]
unseenLemmaIndices_test = np.where(testSetExistence==False)[0]

print('number of unseen data(lemma) in dev set: ', np.count_nonzero(devSetExistence==False))
print('number of unseen data(lemma) in test set: ', np.count_nonzero(testSetExistence==False), end='\n\n')

# compare with (lemma, inflection) pair
  # first: generate (lemma, inflection) pair in sets
trainLemmaInf = np.stack((trainData[0], trainData[3]), axis=-1).tolist()
devLemmaInf = np.stack((devData[0], devData[3]), axis=-1).tolist()
testLemmaInf = np.stack((testData[0], testData[3]), axis=-1).tolist()

devSetExistence = np.array([lemInf in trainLemmaInf for lemInf in devLemmaInf])
testSetExistence = np.array([lemInf in trainLemmaInf for lemInf in testLemmaInf])

unseenLemInfIndices_dev = np.where(devSetExistence==False)[0]
unseenLemInfIndices_test = np.where(testSetExistence==False)[0]

print('number of unseen data((lemma, inflection) pair) in dev set: ', np.count_nonzero(devSetExistence==False))
print('number of unseen data((lemma, inflection) pair) in test set: ', np.count_nonzero(testSetExistence==False))

number of unseen data(lemma) in dev set:  85
number of unseen data(lemma) in test set:  78

number of unseen data((lemma, inflection) pair) in dev set:  108
number of unseen data((lemma, inflection) pair) in test set:  88


In [22]:
# create dataframe for dev set's unseen data
    # 0:lemma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational 
    
data_analysis_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Datasets analysis/'
os.makedirs(data_analysis_path, exist_ok=True)  

unseenLemmaIndices_dev_df = pd.DataFrame({'index': unseenLemmaIndices_dev,
                                           'lemma': np.array(devData[0])[unseenLemmaIndices_dev],
                                           'inflection': np.array(devData[3])[unseenLemmaIndices_dev],
                                           'pl pattern': np.array(devData[4])[unseenLemmaIndices_dev]})
unseenLemmaIndices_dev_df.to_csv(data_analysis_path + 'unseenLemma_dev.csv')


unseenLemInfIndices_dev_df = pd.DataFrame({'index': unseenLemInfIndices_dev,
                                           'lemma': np.array(devData[0])[unseenLemInfIndices_dev],
                                           'inflection': np.array(devData[3])[unseenLemInfIndices_dev],
                                           'pl pattern': np.array(devData[4])[unseenLemInfIndices_dev]})
unseenLemInfIndices_dev_df.to_csv(data_analysis_path + 'unseenLemInf_dev.csv')

In [23]:
unseenLemInfIndices_dev_df.head()

,index,lemma,inflection,pl pattern
0,224,muHaw~il,muHaw~ilAt,mu1aw~i3At
1,234,mujawohar,mujawoharAt,mu1awo3a4At
2,283,Haloy,Huliy~,1u2iy~
3,298,tAriyx,tawAriyx,tawA2iy3
4,325,$Aliyh,$AliyhAt,NTWSAt


In [24]:
# create dataframe for test set's unseen data
unseenLemmaIndices_test_df = pd.DataFrame({'index': unseenLemmaIndices_test,
                                           'lemma': np.array(testData[0])[unseenLemmaIndices_test],
                                           'inflection': np.array(testData[3])[unseenLemmaIndices_test],
                                           'pl pattern': np.array(testData[4])[unseenLemmaIndices_test]})
unseenLemmaIndices_test_df.to_csv(data_analysis_path + 'unseenLemma_test.csv')


unseenLemInfIndices_test_df = pd.DataFrame({'index': unseenLemInfIndices_test,
                                           'lemma': np.array(testData[0])[unseenLemInfIndices_test],
                                           'inflection': np.array(testData[3])[unseenLemInfIndices_test],
                                           'pl pattern': np.array(testData[4])[unseenLemInfIndices_test]})
unseenLemInfIndices_test_df.to_csv(data_analysis_path + 'unseenLemInf_test.csv')

In [25]:
unseenLemInfIndices_test_df.head()

,index,lemma,inflection,pl pattern
0,179,minaS~ap,minaS~At,mi1a2~At
1,274,muDorib,muDoribiyn,mu1o2i3iyn
2,312,Eubuw~ap,Eubuw~At,1u2uw~At
3,339,rafiyqap,rafiyqAt,1a2iy3At
4,362,sAEidap,sawAEid,1awA2i3


## **Work On Duplicates**

In [ ]:
# find duplicates and create dictionaries

def getDuplicateIndexes(data):
  dupindex={}
  f=False
  for i in range(len(data[0])):
    if f: break
    lemma = data[0][i]
    if dupindex.get(lemma)==None:
      dupindex[lemma]=[]
      dupindex[lemma].append(i)
    else:
      f=False
      for k in range(len(dupindex[lemma])):
        if data[8][i] > data[8][dupindex[lemma][k]]:
          dupindex[lemma].insert(0,i)
          f=True
      if not f : dupindex[lemma].append(i)


  return dupindex

def reportErrorAnalysis_duplicates(dataframe,dicDup,path):
  df=dataframe.loc[[]]
  
  indexes=[]
  for key, value in dicDup.items():
    if len(value)>1 :
      for x in value:
        n=len(df)
        indexes.append(x)
        df.loc[n]=dataframe.loc[x]
  df.insert(0, "index", indexes, True)
  df.to_csv(path)
  return df


if is_dataOrginal:
  #Save similar lemmas indexes in dictionary (key->lemma | value->list of indexes)
  trainDupIndex=getDuplicateIndexes(trainData)
  devDupIndex=getDuplicateIndexes(devData)
  testDupIndex=getDuplicateIndexes(testData)
  print("duplicate indexes has been updated")
  print(trainDupIndex)

if is_dataOrginal:
  #Save Duplicate report on data in csv
  trainDuplicateCsv=reportErrorAnalysis_duplicates(train_csv,trainDupIndex,data_analysis_path+"/duplicateAnalyssis_train.csv")
  devDuplicateCsv=reportErrorAnalysis_duplicates(dev_csv,devDupIndex,data_analysis_path+"/duplicateAnalyssis_dev.csv")
  testDuplicateCsv=reportErrorAnalysis_duplicates(test_csv,testDupIndex,data_analysis_path+"/duplicateAnalyssis_test.csv")
  print("Report Error Analysis on duplicates has been saved in /content/drive/MyDrive/AI Projects/Arabic Broken Plural/Datasets analysis path")

#Generate lemma-inflection refrence
dic_lemmaInflection={}
def updateDic_lemmaInflection(Data,dupIndex):
  global dic_lemmaInflection
  for key, value in dupIndex.items():
    if dic_lemmaInflection.get(key)==None:
      dic_lemmaInflection[key]=[]
    for x in value : 
      if not Data[3][x] in dic_lemmaInflection[key]: dic_lemmaInflection[key].append(Data[3][x])

if is_dataOrginal:
  updateDic_lemmaInflection(trainData,trainDupIndex)
  updateDic_lemmaInflection(testData,testDupIndex)
  updateDic_lemmaInflection(devData,devDupIndex)

In [27]:
#Drop unnecessary duplicates
def getUniqueData(data,dupIndex):
  out=[]
  #init nested lists
  for x in data: out.append([])
  #fill out with most frequent lemma-inflection pairs
  for key, value in dupIndex.items():
    for k in range(len(data)):
      out[k].append(data[k][value[0]])
  return out


trainDataUnique=getUniqueData(trainData,trainDupIndex)
devDataUnique=getUniqueData(devData,devDupIndex)
testDataUnique=getUniqueData(testData,testDupIndex)
print("Number Of Lemmas before Dropping duplicates in Train Set: ", len(trainData[0]))
print("Number Of Lemmas before Dropping duplicates in Test Set: ",len(testData[0]))
print("Number Of Lemmas before Dropping duplicates in Dev Set: ",len(devData[0]))
print("=============================================================")
print("Number Of Lemmas After Dropping duplicates in Train Set: ", len(trainDataUnique[0]))
print("Number Of Lemmas After Dropping duplicates in Test Set: ",len(testDataUnique[0]))
print("Number Of Lemmas After Dropping duplicates in Dev Set: ",len(devDataUnique[0]))

Number Of Lemmas before Dropping duplicates in Train Set:  2862
Number Of Lemmas before Dropping duplicates in Test Set:  1103
Number Of Lemmas before Dropping duplicates in Dev Set:  1151
Number Of Lemmas After Dropping duplicates in Train Set:  2475
Number Of Lemmas After Dropping duplicates in Test Set:  1028
Number Of Lemmas After Dropping duplicates in Dev Set:  1060


In [28]:
#######Risky Implementation!!! Only Run This Once
if is_dataOrginal:
  trainData=trainDataUnique
  devData=devDataUnique
  testData=testDataUnique
  is_dataOrginal=False
else:
  print("Warning!!! duplicate rows has already been dropped from data.(skipped this step)")

## **Encoding Datasets (Character-based)**

In [29]:
# generate character encoding
oov_token = 'not available char'
char_index = {oov_token: 0}

def updateCharIndex(data):
  global char_index
  # encode input with encoding character set
  for i in range(len(data[0])):
      chars = data[0][i] + data[1][i] + data[3][i] + data[4][i]
      for c in chars:
          if c not in char_index:
              char_index[c] = len(char_index)

updateCharIndex(trainData)
print(len(char_index))
print(char_index)

50
{'not available char': 0, 'E': 1, 'a': 2, 'm': 3, 'l': 4, '1': 5, '2': 6, '3': 7, '>': 8, 'o': 9, 'A': 10, 'h': 11, 'd': 12, 'f': 13, 'u': 14, 'D': 15, 'w': 16, "'": 17, 'j': 18, 'i': 19, 'z': 20, 'p': 21, 'H': 22, 'k': 23, 'n': 24, 'S': 25, 'r': 26, '4': 27, 'q': 28, '~': 29, 'y': 30, 't': 31, 's': 32, '<': 33, 'b': 34, 'v': 35, '|': 36, 'T': 37, '}': 38, 'Z': 39, '{': 40, '&': 41, '$': 42, 'x': 43, 'N': 44, 'W': 45, '*': 46, 'g': 47, 'Y': 48, 'F': 49}


In [30]:
# encode words with character encoder
def encodeWord(word, dic=char_index, oov=oov_token):
    return [dic[c] if c in dic else dic[oov] for c in list(word)]

# encode data and get maximum length of features
def getEncodedData(data):
  '''
  Output type->List : 0:lamma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational | 8:number of root
  '''

  data_encoded = data.copy()
  data_encoded[0] = [encodeWord(each) for each in data[0]]
  data_encoded[1] = [encodeWord(each) for each in data[1]]
  data_encoded[2] = [encodeWord(each) for each in data[2]]
  data_encoded[3] = [encodeWord(each) for each in data[3]]
  data_encoded[4] = [encodeWord(each) for each in data[4]]

  lemma_max_len = max([len(each) for each in data_encoded[0]])
  root_max_len = max([len(each) for each in data_encoded[2]])
  inflection_max_len = max([len(each) for each in data_encoded[3]])

  return data_encoded, lemma_max_len, root_max_len, inflection_max_len


trainData_encoded, lemma_max_len, root_max_len, inflection_max_len = getEncodedData(trainData)
devData_encoded, _, _, _  = getEncodedData(devData)
testData_encoded, _, _, _ = getEncodedData(testData)

print('lemma shape train: ', len(trainData_encoded[0]))
print('lemma shape dev: ', len(devData_encoded[0]))
print('lemma shape test: ', len(testData_encoded[0]))
print('max length in lemma elements: ', lemma_max_len)
print('max length in root elements: ', root_max_len)
print('max length in inflection elements: ', inflection_max_len)

lemma shape train:  2475
lemma shape dev:  1060
lemma shape test:  1028
max length in lemma elements:  17
max length in root elements:  4
max length in inflection elements:  18


In [31]:
# ADD padding to sample
# 0:lamma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational 
def getPaddedData(encodedData, lemma_max_len, root_max_len, inflection_max_len):
  data_pad = []
  data_pad.append(np.array(pad_sequences(encodedData[0], maxlen=lemma_max_len, padding='post'))) 
  data_pad.append(np.array(pad_sequences(encodedData[1], maxlen=lemma_max_len, padding='post')))
  data_pad.append(np.array(pad_sequences(encodedData[2], maxlen=root_max_len, padding='post')))
  data_pad.append(np.array(pad_sequences(encodedData[3], maxlen=inflection_max_len, padding='post')))
  data_pad.append(np.array(pad_sequences(encodedData[4], maxlen=inflection_max_len, padding='post')))
  data_pad.append(np.array(encodedData[5]))
  data_pad.append(np.array(encodedData[6]))
  data_pad.append(np.array(encodedData[7]))
  data_pad.append(np.array(encodedData[8]))

  return data_pad

trainData_pad = getPaddedData(trainData_encoded, lemma_max_len, root_max_len, inflection_max_len)
devData_pad = getPaddedData(devData_encoded, lemma_max_len, root_max_len, inflection_max_len)
testData_pad = getPaddedData(testData_encoded, lemma_max_len, root_max_len, inflection_max_len)

print('lemma with pad shape train: ', trainData_pad[0].shape)
print('lemma with pad shape dev: ', devData_pad[0].shape)
print('lemma with pad shape test: ', testData_pad[0].shape)

lemma with pad shape train:  (2475, 17)
lemma with pad shape dev:  (1060, 17)
lemma with pad shape test:  (1028, 17)


### **Create inputs of neural network**

In [32]:
# 0:lamma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational | 8:number of root
# create train set
X_train_lemma = trainData_pad[0]  # lemma feature
X_train_singularPattern = trainData_pad[1]  # singular pattern feature
X_train_root = trainData_pad[2]  # root feature
X_train_genRat = np.stack((trainData_pad[6], trainData_pad[7]), axis=-1)  # gender & rational features
X_train_numOfRoot = trainData_pad[8].reshape((trainData_pad[8].shape[0], -1))  # number of root

y_train_class = trainData_pad[5].reshape((trainData_pad[5].shape[0], -1))  # broken or sound classes
y_train_plPattern = trainData_pad[4]  # plural pattern
y_train_inflection = trainData_pad[3]  # inflection

# create dev set
X_dev_lemma = devData_pad[0]  # lemma feature
X_dev_singularPattern = devData_pad[1]  # singular pattern feature
X_dev_root = devData_pad[2]  # root feature
X_dev_genRat = np.stack((devData_pad[6], devData_pad[7]), axis=-1)  # gender & rational features
X_dev_numOfRoot = devData_pad[8].reshape((devData_pad[8].shape[0], -1))  # number of root

y_dev_class = devData_pad[5].reshape((devData_pad[5].shape[0], -1))  # broken or sound classes
y_dev_plPattern = devData_pad[4]  # plural pattern
y_dev_inflection = devData_pad[3]  # inflection

# create test set
X_test_lemma = testData_pad[0]  # lemma feature
X_test_singularPattern = testData_pad[1]  # singular pattern feature
X_test_root = testData_pad[2]  # root feature
X_test_genRat = np.stack((testData_pad[6], testData_pad[7]), axis=-1)  # gender & rational features
X_test_numOfRoot = testData_pad[8].reshape((testData_pad[8].shape[0], -1))  # number of root

y_test_class = testData_pad[5].reshape((testData_pad[5].shape[0], -1))  # broken or sound classes
y_test_plPattern = testData_pad[4]  # plural pattern
y_test_inflection = testData_pad[3]  # inflection

print('X_train_lemma shape: ', X_train_lemma.shape)
print('X_train_singularPattern shape: ', X_train_singularPattern.shape)
print('X_train_root shape: ', X_train_root.shape)
print('X_train_genRat shape: ', X_train_genRat.shape)
print('X_train_numOfRoot shape: ', X_train_numOfRoot.shape)
print('y_train_class shape: ', y_train_class.shape)
print('y_train_plPattern shape: ', y_train_plPattern.shape)
print('y_train_unflection shape: ', y_train_inflection.shape)

X_train_lemma shape:  (2475, 17)
X_train_singularPattern shape:  (2475, 17)
X_train_root shape:  (2475, 4)
X_train_genRat shape:  (2475, 2)
X_train_numOfRoot shape:  (2475, 1)
y_train_class shape:  (2475, 1)
y_train_plPattern shape:  (2475, 18)
y_train_unflection shape:  (2475, 18)


In [33]:
# Convert to ONE-HOT encoding
X_train_lemma_OH = to_categorical(X_train_lemma, len(char_index))
X_train_singularPattern_OH = to_categorical(X_train_singularPattern, len(char_index))
X_train_root_OH = to_categorical(X_train_root, len(char_index))

X_dev_lemma_OH = to_categorical(X_dev_lemma, len(char_index))
X_dev_singularPattern_OH = to_categorical(X_dev_singularPattern, len(char_index))
X_dev_root_OH = to_categorical(X_dev_root, len(char_index))

X_test_lemma_OH = to_categorical(X_test_lemma, len(char_index))
X_test_singularPattern_OH = to_categorical(X_test_singularPattern, len(char_index))
X_test_root_OH = to_categorical(X_test_root, len(char_index))

y_train_plPattern_OH = to_categorical(y_train_plPattern, len(char_index))
y_dev_plPattern_OH = to_categorical(y_dev_plPattern, len(char_index))
y_test_plPattern_OH = to_categorical(y_test_plPattern, len(char_index))

y_train_inflection_OH = to_categorical(y_train_inflection, len(char_index))
y_dev_inflection_OH = to_categorical(y_dev_inflection, len(char_index))
y_test_inflection_OH = to_categorical(y_test_inflection, len(char_index))

print(X_train_lemma_OH.shape)
print(X_train_singularPattern_OH.shape)
print(X_train_root_OH.shape)
print(y_train_plPattern_OH.shape)
print(y_train_inflection_OH.shape)

(2475, 17, 50)
(2475, 17, 50)
(2475, 4, 50)
(2475, 18, 50)
(2475, 18, 50)


In [34]:
# covern numerical inputs to int32
X_train_genRat = np.asarray(X_train_genRat).astype('int32')
X_train_numOfRoot = np.asarray(X_train_numOfRoot).astype('int32')
X_dev_genRat = np.asarray(X_dev_genRat).astype('int32')
X_dev_numOfRoot = np.asarray(X_dev_numOfRoot).astype('int32')
X_test_genRat = np.asarray(X_test_genRat).astype('int32')
X_test_numOfRoot = np.asarray(X_test_numOfRoot).astype('int32')

y_train_class = np.asarray(y_train_class).astype('int32')
y_dev_class = np.asarray(y_dev_class).astype('int32')
y_test_class = np.asarray(y_test_class).astype('int32')

## **Encoding Bert Inputs**

In [ ]:
max_len = 8

def getBertInputs(data,max_len):
  ids=[]
  token_type_ids=[]
  attention_mask=[]

  for i in range(len(data[0])):
    bert_tokenized = bert_tokenizer.encode_plus(
        ''.join(getArabicForm(data[0][i])),
        max_length=max_len,
        add_special_tokens=True, # Add '[CLS]' and '[SEP]'
        return_token_type_ids=True,
        padding='max_length',  # Pad to longest in batch.
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'  # Return PyTorch tensors
    )
    ids.append(bert_tokenized['input_ids'])
    token_type_ids.append(bert_tokenized['token_type_ids'])
    attention_mask.append(bert_tokenized['attention_mask'])
  
  return ids,token_type_ids,attention_mask

trainBert_ids, trainBert_token_type, trainBert_attention_mask = getBertInputs(trainData, max_len)
devBert_ids, devBert_token_type, devBert_attention_mask = getBertInputs(devData, max_len)
testBert_ids, testBert_token_type, testBert_attention_mask = getBertInputs(testData, max_len)

In [ ]:
print(min([np.count_nonzero(x==0) for x in trainBert_ids]))

2


In [ ]:
encoder = TFBertModel.from_pretrained("asafaya/bert-base-arabic")

def getEmbeding(ids,token_type,attention_mask,max_len):  
  bert_embedding=[]
  for i in range(len(ids)):
    embedding_output = np.array(encoder(input_ids=ids[i], token_type_ids=token_type[i], attention_mask=attention_mask[i])[1])
    embedding_output = embedding_output.reshape(embedding_output.shape[1])
    bert_embedding.append(embedding_output)

  bert_embedding = np.stack(bert_embedding, axis=0)
  return bert_embedding


with tf.device('/GPU:0'):
  trainBert_embedding = getEmbeding(trainBert_ids, trainBert_token_type, trainBert_attention_mask, max_len)
  np.save(dataPath+'/train_bert_embedding.npy', trainBert_embedding)
  print('train is done!')
  devBert_embedding = getEmbeding(devBert_ids, devBert_token_type, devBert_attention_mask, max_len)
  np.save(dataPath+'/dev_bert_embedding.npy', devBert_embedding)
  print('dev is done!')
  testBert_embedding = getEmbeding(testBert_ids, testBert_token_type, testBert_attention_mask, max_len)
  np.save(dataPath+'/test_bert_embedding.npy', testBert_embedding)
  print('test is done!')


In [35]:
# read bert_embedding from google drive
trainBert_embedding = np.load(dataPath+'/train_bert_embedding.npy')
devBert_embedding = np.load(dataPath+'/dev_bert_embedding.npy')
testBert_embedding = np.load(dataPath+'/test_bert_embedding.npy')

In [36]:
print(trainBert_embedding.shape)
print(devBert_embedding.shape)
print(testBert_embedding.shape)

(2475, 768)
(1060, 768)
(1028, 768)


### **Create inputs of neural network**

In [37]:
# 0:lamma | 1:singular pattern | 2:root |3:inflection | 4:plural pattern | 5:plural type(B/S) | 6:gender | 7:rational | 8:number of root
# create train set
X_train_bert = trainBert_embedding
X_train_lemma = trainData_pad[0]  # lemma feature
X_train_singularPattern = trainData_pad[1]  # singular pattern feature
X_train_root = trainData_pad[2]  # root feature
X_train_genRat = np.stack((trainData_pad[6], trainData_pad[7]), axis=-1)  # gender & rational features
X_train_numOfRoot = trainData_pad[8].reshape((trainData_pad[8].shape[0], -1))  # number of root

y_train_class = trainData_pad[5].reshape((trainData_pad[5].shape[0], -1))  # broken or sound classes
y_train_plPattern = trainData_pad[4]  # plural pattern
y_train_inflection = trainData_pad[3]  # inflection

# create dev set
X_dev_bert = devBert_embedding
X_dev_lemma = devData_pad[0]  # lemma feature
X_dev_singularPattern = devData_pad[1]  # singular pattern feature
X_dev_root = devData_pad[2]  # root feature
X_dev_genRat = np.stack((devData_pad[6], devData_pad[7]), axis=-1)  # gender & rational features
X_dev_numOfRoot = devData_pad[8].reshape((devData_pad[8].shape[0], -1))  # number of root

y_dev_class = devData_pad[5].reshape((devData_pad[5].shape[0], -1))  # broken or sound classes
y_dev_plPattern = devData_pad[4]  # plural pattern
y_dev_inflection = devData_pad[3]  # inflection

# create test set
X_test_bert = testBert_embedding
X_test_lemma = testData_pad[0]  # lemma feature
X_test_singularPattern = testData_pad[1]  # singular pattern feature
X_test_root = testData_pad[2]  # root feature
X_test_genRat = np.stack((testData_pad[6], testData_pad[7]), axis=-1)  # gender & rational features
X_test_numOfRoot = testData_pad[8].reshape((testData_pad[8].shape[0], -1))  # number of root

y_test_class = testData_pad[5].reshape((testData_pad[5].shape[0], -1))  # broken or sound classes
y_test_plPattern = testData_pad[4]  # plural pattern
y_test_inflection = testData_pad[3]  # inflection

print('X_train_bert shape: ', X_train_bert.shape)
print('X_train_lemma shape: ', X_train_lemma.shape)
print('X_train_singularPattern shape: ', X_train_singularPattern.shape)
print('X_train_root shape: ', X_train_root.shape)
print('X_train_genRat shape: ', X_train_genRat.shape)
print('X_train_numOfRoot shape: ', X_train_numOfRoot.shape)
print('y_train_class shape: ', y_train_class.shape)
print('y_train_plPattern shape: ', y_train_plPattern.shape)
print('y_train_unflection shape: ', y_train_inflection.shape)

X_train_bert shape:  (2475, 768)
X_train_lemma shape:  (2475, 17)
X_train_singularPattern shape:  (2475, 17)
X_train_root shape:  (2475, 4)
X_train_genRat shape:  (2475, 2)
X_train_numOfRoot shape:  (2475, 1)
y_train_class shape:  (2475, 1)
y_train_plPattern shape:  (2475, 18)
y_train_unflection shape:  (2475, 18)


In [38]:
# Convert to ONE-HOT encoding
X_train_lemma_OH = to_categorical(X_train_lemma, len(char_index))
X_train_singularPattern_OH = to_categorical(X_train_singularPattern, len(char_index))
X_train_root_OH = to_categorical(X_train_root, len(char_index))

X_dev_lemma_OH = to_categorical(X_dev_lemma, len(char_index))
X_dev_singularPattern_OH = to_categorical(X_dev_singularPattern, len(char_index))
X_dev_root_OH = to_categorical(X_dev_root, len(char_index))

X_test_lemma_OH = to_categorical(X_test_lemma, len(char_index))
X_test_singularPattern_OH = to_categorical(X_test_singularPattern, len(char_index))
X_test_root_OH = to_categorical(X_test_root, len(char_index))

y_train_plPattern_OH = to_categorical(y_train_plPattern, len(char_index))
y_dev_plPattern_OH = to_categorical(y_dev_plPattern, len(char_index))
y_test_plPattern_OH = to_categorical(y_test_plPattern, len(char_index))

y_train_inflection_OH = to_categorical(y_train_inflection, len(char_index))
y_dev_inflection_OH = to_categorical(y_dev_inflection, len(char_index))
y_test_inflection_OH = to_categorical(y_test_inflection, len(char_index))

print(X_train_lemma_OH.shape)
print(X_train_singularPattern_OH.shape)
print(X_train_root_OH.shape)
print(y_train_plPattern_OH.shape)
print(y_train_inflection_OH.shape)

(2475, 17, 50)
(2475, 17, 50)
(2475, 4, 50)
(2475, 18, 50)
(2475, 18, 50)


In [39]:
# convert numerical inputs to int32
X_train_genRat = np.asarray(X_train_genRat).astype('int32')
X_train_numOfRoot = np.asarray(X_train_numOfRoot).astype('int32')
X_dev_genRat = np.asarray(X_dev_genRat).astype('int32')
X_dev_numOfRoot = np.asarray(X_dev_numOfRoot).astype('int32')
X_test_genRat = np.asarray(X_test_genRat).astype('int32')
X_test_numOfRoot = np.asarray(X_test_numOfRoot).astype('int32')

y_train_class = np.asarray(y_train_class).astype('int32')
y_dev_class = np.asarray(y_dev_class).astype('int32')
y_test_class = np.asarray(y_test_class).astype('int32')

# **Helper Functions**

In [40]:
def getDuplicateIndexes(data):
  dupindex={}
  for i in range(len(data[0])):
    if dupindex.get(data[0][i])==None:
      dupindex[data[0][i]]=[]
    dupindex[data[0][i]].append(i)

  return dupindex

def reportErrorAnalysis_duplicates(dataframe,dicDup,path):
  df=dataframe.loc[[]]
  
  indexes=[]
  for key, value in dicDup.items():
    for x in value:
      n=len(df)
      indexes.append(x)
      df.loc[n]=dataframe.loc[x]
  df.insert(0, "index", indexes, True)
  df.to_csv(path)
  return df


#Save similar lemmas indexes in dictionary (key->lemma | value->list of indexes)
trainDupIndex=getDuplicateIndexes(trainData)
devDupIndex=getDuplicateIndexes(devData)
testDupIndex=getDuplicateIndexes(testData)


#Save Duplicate report on data in csv
trainDuplicateCsv=reportErrorAnalysis_duplicates(train_csv,trainDupIndex,dataPath+"/duplicateAnalyssis_train.csv")
devDuplicateCsv=reportErrorAnalysis_duplicates(dev_csv,devDupIndex,dataPath+"/duplicateAnalyssis_dev.csv")
testDuplicateCsv=reportErrorAnalysis_duplicates(test_csv,testDupIndex,dataPath+"/duplicateAnalyssis_test.csv")

#Generate lemma-inflection refrence
dic_lemmaInflection={}
def updateDic_lemmaInflection(Data,dupIndex):
  global dic_lemmaInflection
  for key, value in dupIndex.items():
    if dic_lemmaInflection.get(key)==None:
      dic_lemmaInflection[key]=[]
      for x in value : 
        if not Data[3][x] in dic_lemmaInflection[key]: dic_lemmaInflection[key].append(Data[3][x])

updateDic_lemmaInflection(trainData,trainDupIndex)
updateDic_lemmaInflection(testData,testDupIndex)
updateDic_lemmaInflection(devData,devDupIndex)

In [41]:
updateDic_lemmaInflection

<function __main__.updateDic_lemmaInflection(Data, dupIndex)>

In [42]:
def oneHot2D(arr):
    tmp = np.zeros_like(arr)
    tmp[np.arange(len(arr)), arr.argmax(1)] = 1
    return tmp

def oneHot3D(arr):
    tmp = np.array([oneHot2D(x) for x in arr])
    return tmp

In [43]:
char_index_key_list = list(char_index.keys())
char_index_val_list = list(char_index.values())
def getCharFromIdx(idx):
  position = char_index_val_list.index(idx)
  return char_index_key_list[position]

In [44]:
def getWordFromOnehot(onehot):
  encoded=np.argmax(onehot, axis=1)
  word=[]
  for x in encoded:
    if x==0:break
    word.append(getCharFromIdx(x))
  return ''.join(word)

def binaryConversion(arr, threshold=0.5):
  tmp = np.zeros_like(arr)
  tmp[arr > threshold] = 1
  return tmp

def reportResult(model, history, X_test, y_test):

    # best models based on acc or loss in tarin set or test set
    testHistory = list(
        map(lambda x, y: [x, y], history['val_accuracy'], history['val_loss']))
    print(
        f"best model based on min test set loss:  acc= {min(testHistory, key = lambda k: k[1])[0]}  loss= {min(testHistory, key = lambda k: k[1])[1]}")
    print(
        f"best model based on max test set accuracy:  acc= {max(testHistory, key = lambda k: k[0])[0]}  loss= {max(testHistory, key = lambda k: k[0])[1]}")

    print("\nevaluate dataset with best model based on maximum test set accuracy")
    print("evaluate test set = ", model.evaluate(X_test, y_test, verbose=0))

    y_test_pred = model.predict(X_test)
    y_test_pred = binaryConversion(y_test_pred)

    # confusion matrix and precision, recall and f1 report
    print('-'*30, 'metrics for test set', '-'*30)
    print("confusion matrix: \n", metrics.confusion_matrix(y_test, y_test_pred))
    print(metrics.classification_report(y_test,
          y_test_pred, digits=3, target_names=['Broken', 'Sound']))
    
def argmaxKeepDimensions(arr):
  tmp = np.zeros_like(arr)
  tmp[np.arange(len(arr)), arr.argmax(1)] = 1
  return tmp

def checkAccuracy(pred, true):
  size = pred.shape[0]
  numOfTrue = 0
  for (x, y) in zip(pred, true):
    if np.array_equal(x, y):
      numOfTrue += 1
  
  return size-numOfTrue, numOfTrue/size

def checkAccuracy_anyAcceptableInflectionInDataset(lemma, pred, dic_refrence):
  size = pred.shape[0]
  numOfTrue = 0
  for i in range (len(pred)):
    w=getWordFromOnehot(pred[i])
    if w in dic_refrence[lemma[i]]:
      numOfTrue += 1
  
  return size-numOfTrue, numOfTrue/size

def reportErrorAnalysis(lemma, bs, pred_OH, inflection_OH, path):
  size = pred_OH.shape[0]
  report={'index':[], 'lemma':[], 'b/s':[], 'inflection':[], 'predict':[]}
  i=0
  for (x, y) in zip(pred_OH, inflection_OH):
    if not np.array_equal(x, y):
      report['index'].append(i)
      report['lemma'].append(lemma[i])
      report['b/s'].append(bs[i])
      report['inflection'].append(getWordFromOnehot(inflection_OH[i]))
      report['predict'].append(getWordFromOnehot(pred_OH[i]))
    i+=1
  
  df = pd.DataFrame(report) 
  df.to_csv(path)
  return df

def reportErrorAnalysis_inflectionWithBS(lemma, bs,bs_pred, pred_OH, inflection_OH, path):
  size = pred_OH.shape[0]
  report={'index':[], 'lemma':[], 'b/s':[], 'b/s predict':[], 'inflection':[], 'predict':[]}
  bs_pred = binaryConversion(bs_pred)
  i=0
  for (x, y) in zip(pred_OH, inflection_OH):
    if not np.array_equal(x, y):
      report['index'].append(i)
      report['lemma'].append(lemma[i])
      report['b/s'].append(bs[i])
      report['b/s predict'].append(bs_pred[i])
      report['inflection'].append(getWordFromOnehot(inflection_OH[i]))
      report['predict'].append(getWordFromOnehot(pred_OH[i]))
    i+=1
  
  df = pd.DataFrame(report) 
  df.to_csv(path)
  return df

def reportErrorAnalysis_classification(lemma, inflection_OH, bs, bs_pred, path):
  size = bs.shape[0]
  report={'index':[], 'lemma':[], 'inflection':[],'b/s':[], 'b/s predict':[] }
  bs_pred = binaryConversion(bs_pred, 0.5)
  i=0
  for (x, y) in zip(bs, bs_pred):
    if not x==y:
      report['index'].append(i)
      report['lemma'].append(lemma[i])
      report['b/s'].append(x)
      report['inflection'].append(getWordFromOnehot(inflection_OH[i]))
      report['b/s predict'].append(y)
    i+=1
  
  df = pd.DataFrame(report) 
  df.to_csv(path)
  return df

In [45]:
def reportPercentageOfBrokenSoundErrors(dataset_errorAnalysis):
  total_errors_num = dataset_errorAnalysis.shape[0]
  train_sound_error = np.sum(dataset_errorAnalysis['b/s'])
  train_broken_error = total_errors_num - train_sound_error
  return train_broken_error/total_errors_num * 100, train_sound_error/total_errors_num * 100

def reportPercentageOfUnseenErrors(errorIndices, unseenIndices):
  isInUnseen = np.array([index in unseenIndices for index in errorIndices])
  return np.count_nonzero(isInUnseen==True) / len(errorIndices) * 100, np.count_nonzero(isInUnseen==True) / len(unseenIndices) * 100

# **Classification Models**

In [ ]:
path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Classification Models'
# path = '/content/drive/MyDrive/Deep Learning/Datasets/Arabic Broken Plural/Results/Classification Models'

In [ ]:
lemma_length = X_train_lemma_OH.shape[1] #17
encode_size = X_train_lemma_OH.shape[2] #50
genRat_length = X_train_genRat.shape[1] #2

epochs = 100
batch_size = 64

## **Model 1**

In [ ]:
input_lemma = Input(shape=(lemma_length, encode_size), name='Lemma')
input_genRat = Input(shape=(genRat_length,), name='Gender-Rational')


lstm = Bidirectional(LSTM(256, name='Lstm'), name='Bidirectional')(input_lemma)

c = Concatenate(axis=1, name='Concatenate')([lstm, input_genRat])

d = Dense(256, activation='relu', name='Dense1')(c)
d = Dense(64, activation='relu', name='Dense2')(d)

output = Dense(1, activation='sigmoid', name='Class')(d)
model = Model(inputs=[input_lemma, input_genRat], outputs=output)

model.save(path + "/model 1/structure.h5")

checkpoint_filepath = path + '/model 1/weights.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True, verbose=0)

model.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
  history = model.fit([X_train_lemma_OH, X_train_genRat], y_train_class, batch_size=batch_size, epochs=epochs, 
                      verbose=1, callbacks=[checkpoint], validation_data=([X_dev_lemma_OH, X_dev_genRat], y_dev_class)).history

In [ ]:
print(model.summary())
plot_model(model, to_file=path+'/model 1/architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
model = load_model(checkpoint_filepath)
model.evaluate([X_test_lemma_OH, X_test_genRat], y_test_class)

In [ ]:
reportResult(model, history, [X_test_lemma_OH, X_test_genRat], y_test_class)

### **Error Analysis**

Train set

In [ ]:
train_class_p=model.predict([X_train_lemma_OH, X_train_genRat])
reportErrorAnalysis_classification(trainData[0], y_train_inflection_OH, y_train_class, train_class_p, path + "/model 1/train_errorAnalysis.csv")

Dev set

In [ ]:
dev_class_p=model.predict([X_dev_lemma_OH, X_dev_genRat])
reportErrorAnalysis_classification(devData[0], y_dev_inflection_OH, y_dev_class, dev_class_p, path + "/model 1/dev_errorAnalysis.csv")

Test set

In [ ]:
test_class_p=model.predict([X_test_lemma_OH, X_test_genRat])
reportErrorAnalysis_classification(testData[0], y_test_inflection_OH, y_test_class, test_class_p, path + "/model 1/test_errorAnalysis.csv")

# **Machine Translation Models**

In [ ]:
path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Machine Translation Models'
# path = '/content/drive/MyDrive/Deep Learning/Datasets/Arabic Broken Plural/Results/Machine Translation Models'

In [ ]:
bert_embedding_length = X_train_bert.shape[1] #768
lemma_length = X_train_lemma_OH.shape[1] #17
singularPattern_length = X_train_singularPattern_OH.shape[1] #17
root_length = X_train_root_OH.shape[1] #4
genRat_length = X_train_genRat.shape[1] #2
numOfRoot_length = X_train_numOfRoot.shape[1] #1
plPattern_length = y_train_plPattern_OH.shape[1] #18
inflection_length = y_train_inflection_OH.shape[1] #18
encode_size = X_train_lemma_OH.shape[2] #50

epochs = 100
batch_size = 64

## **Model 1**
lemma - inflection

In [ ]:
model1_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Machine Translation Models/Model 1'

In [ ]:
input_lemma = Input(shape=(lemma_length, encode_size), name='Lemma')
input_genRat = Input(shape=(genRat_length, ), name='Gender-Rational')


lstm1 = Bidirectional(LSTM(256, name='Lstm1'), name='Encoder')(input_lemma)

c = Concatenate(axis=1, name='Concatenate')([lstm1, input_genRat])

d = Dense(256, activation='relu', name='Dense1')(c)

r = RepeatVector(inflection_length, name='RepeatVector')(d)

lstm2 = Bidirectional(LSTM(256, return_sequences=True, name='Lstm2'), name='Decoder')(r)

output = TimeDistributed(Dense(encode_size, activation='softmax', name='Dense2'), name='Inflection')(lstm2)

model1 = Model(inputs=[input_lemma, input_genRat], outputs=output)

checkpoint_filepath = model1_path + '/weights.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True, verbose=0)

model1.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
  model1.fit([X_train_lemma_OH, X_train_genRat], y_train_inflection_OH, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=([X_dev_lemma_OH, X_dev_genRat], y_dev_inflection_OH))  # starts training

In [ ]:
model1 = load_model(checkpoint_filepath)
model1.evaluate([X_test_lemma_OH, X_test_genRat], y_test_inflection_OH)

In [ ]:
print(model1.summary())
plot_model(model1, to_file=model1_path+'/architecture.png', show_shapes=True, show_layer_names=True)

### **Train analyze**

In [ ]:
train_pred = model1.predict([X_train_lemma_OH, X_train_genRat])
train_pred_OH = oneHot3D(train_pred)
train_error_num, train_acc_num = checkAccuracy(train_pred_OH, y_train_inflection_OH)
train_error_num, train_acc_num

In [ ]:
y_train_inflection_OH.shape

In [ ]:
train_errorAnalysis = reportErrorAnalysis(trainData[0], trainData[5], train_pred_OH, y_train_inflection_OH, model1_path + "/train_errorAnalysis.csv")
train_errorAnalysis.head()

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(train_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(trainData[0],train_pred_OH,dic_lemmaInflection)

### **Dev analyze**

In [ ]:
dev_pred = model1.predict([X_dev_lemma_OH, X_dev_genRat])
dev_pred_OH = oneHot3D(dev_pred)
dev_error_num, dev_acc_num = checkAccuracy(dev_pred_OH, y_dev_inflection_OH)
dev_error_num, dev_acc_num

In [ ]:
dev_errorAnalysis = reportErrorAnalysis(devData[0], devData[5], dev_pred_OH, y_dev_inflection_OH, model1_path + "/dev_errorAnalysis.csv")
dev_errorAnalysis.head()

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(dev_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(devData[0],dev_pred_OH,dic_lemmaInflection)

In [ ]:
# Report Percentage of Unseen Errors in Dev set
errorIndices_dev = dev_errorAnalysis['index'].tolist()
unseenIndices_dev = unseenLemInfIndices_dev_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_dev, unseenIndices_dev))

### **Test analyze**

In [ ]:
test_pred = model1.predict([X_test_lemma_OH, X_test_genRat])
test_pred_OH = oneHot3D(test_pred)
test_error_num, test_acc_num = checkAccuracy(test_pred_OH, y_test_inflection_OH)
test_error_num, test_acc_num

In [ ]:
test_errorAnalysis = reportErrorAnalysis(testData[0], testData[5], test_pred_OH, y_test_inflection_OH, model1_path + "/test_errorAnalysis.csv")

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(test_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(testData[0],test_pred_OH,dic_lemmaInflection)

In [ ]:
# Report Percentage of Unseen Errors in Test set
errorIndices_test = test_errorAnalysis['index'].tolist()
unseenIndices_test = unseenLemInfIndices_test_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_test, unseenIndices_test))

## **Mix Model (2 Networks: Classification + Machine Translation)**
**with Bert embedding input**

In [ ]:
mixModel1_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Machine Translation Models/Mix Model 1'

In [ ]:
BSClassification_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Classification Models'
BSClassification_model_path = BSClassification_path + '/model 1/weights.hdf5'
BSClassification_model_structure_path = BSClassification_path + "/model 1/structure.h5"

BSC_model = load_model(BSClassification_model_path)
BSC_model_struct = load_model(BSClassification_model_structure_path)

BSC_model_struct.set_weights(BSC_model.get_weights())
BSC_model_struct.trainable = False
# print(BSC_model.layers[1].get_weights()[0][0])
# print(BSC_model_struct.layers[1].get_weights()[0][0])

In [ ]:
imput_bert_embedding = Input(shape=(bert_embedding_length, ), name='Bert-Embedding')
input_lemma = Input(shape=(lemma_length, encode_size), name='Lemma')
input_genRat = Input(shape=(genRat_length, ), name='Gender-Rational')

cls = BSC_model_struct([input_lemma, input_genRat])
lstm1 = Bidirectional(LSTM(256, name='Lstm1'), name='Encoder')(input_lemma)
d = Dense(256,  activation='relu', name='Dense1')(imput_bert_embedding)

c = Concatenate(axis=1, name='Concatenate')([lstm1, input_genRat, cls, d])

d = Dense(200, activation='relu', name='Dense2')(c)
r = RepeatVector(inflection_length, name='RepeatVector')(d)

lstm2 = Bidirectional(LSTM(256, return_sequences=True, name='Lstm2'), name='Decoder')(r)
output = TimeDistributed(Dense(encode_size, activation='softmax', name='Dense2'), name='Inflection')(lstm2)

mix_model = Model(inputs=[imput_bert_embedding, input_lemma, input_genRat], outputs=output)

checkpoint_filepath = mixModel1_path + '/weights.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True, verbose=0)

mix_model.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
mix_model.layers[4].layers[1].get_weights()[0][0]

array([-0.02294529, -0.10742611,  0.02736146, ..., -0.18890691,
       -0.01805112, -0.01145793], dtype=float32)

In [ ]:
with tf.device('/GPU:0'):
  mix_model.fit([X_train_bert, X_train_lemma_OH, X_train_genRat], y_train_inflection_OH, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=([X_dev_bert, X_dev_lemma_OH, X_dev_genRat], y_dev_inflection_OH))  # starts training

Epoch 1/100
39/39 [==============================] - 12s 92ms/step - loss: 0.0477 - accuracy: 0.5686 - val_loss: 0.0391 - val_accuracy: 0.6088
Epoch 2/100
39/39 [==============================] - 1s 26ms/step - loss: 0.0384 - accuracy: 0.6188 - val_loss: 0.0368 - val_accuracy: 0.6297
Epoch 3/100
39/39 [==============================] - 1s 24ms/step - loss: 0.0366 - accuracy: 0.6412 - val_loss: 0.0361 - val_accuracy: 0.6310
Epoch 4/100
39/39 [==============================] - 1s 25ms/step - loss: 0.0352 - accuracy: 0.6576 - val_loss: 0.0346 - val_accuracy: 0.6562
Epoch 5/100
39/39 [==============================] - 1s 24ms/step - loss: 0.0338 - accuracy: 0.6759 - val_loss: 0.0329 - val_accuracy: 0.7026
Epoch 6/100
39/39 [==============================] - 1s 24ms/step - loss: 0.0328 - accuracy: 0.6890 - val_loss: 0.0315 - val_accuracy: 0.7120
Epoch 7/100
39/39 [==============================] - 1s 20ms/step - loss: 0.0313 - accuracy: 0.7112 - val_loss: 0.0312 - val_accuracy: 0.7070
Epoch

In [ ]:
mix_model = load_model(checkpoint_filepath)
mix_model.evaluate([X_test_bert, X_test_lemma_OH, X_test_genRat], y_test_inflection_OH)

33/33 [==============================] - 2s 9ms/step - loss: 0.0037 - accuracy: 0.9843


[0.003724090987816453, 0.9843277335166931]

In [ ]:
print(mix_model.summary())
plot_model(mix_model, to_file=mixModel1_path+'/architecture.png', show_shapes=True, show_layer_names=True)

### **Train analyze**

In [ ]:
train_pred = mix_model.predict([X_train_bert, X_train_lemma_OH, X_train_genRat])
train_pred_OH = oneHot3D(train_pred)
train_error_num, train_acc_num = checkAccuracy(train_pred_OH, y_train_inflection_OH)
train_error_num, train_acc_num

78/78 [==============================] - 3s 7ms/step


(10, 0.9959595959595959)

In [ ]:
y_train_inflection_OH.shape

(2475, 18, 50)

In [ ]:
train_errorAnalysis = reportErrorAnalysis(trainData[0], trainData[5], train_pred_OH, y_train_inflection_OH, mixModel1_path + "/train_errorAnalysis.csv")
train_errorAnalysis.head()

,index,lemma,b/s,inflection,predict
0,863,tilofizyuwn,1,tilofizyuwnAt,tilofizzuwnAt
1,939,fiyliyb~iyniy~,1,fiyliyb~iyniy~iyn,fiyliyb~iyiiyiyn
2,1045,<isotuwdiyuw,1,<isotuwdiyuwAt,<isotuwdiywAt
3,1315,frAnokuwfuwniy~,1,franokuwfuwniy~iyn,fannokuwfuwniy~iyn
4,1340,>usoturAliy~,1,>uwsoturAliy~iyn,>uwsourrAliy~iyn


In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(train_errorAnalysis))

(Broken error %, Sound error %):  (10.0, 90.0)


In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(trainData[0],train_pred_OH,dic_lemmaInflection)

(10, 0.9959595959595959)

### **Dev analyze**

In [ ]:
dev_pred = mix_model.predict([X_dev_bert, X_dev_lemma_OH, X_dev_genRat])
dev_pred_OH = oneHot3D(dev_pred)
dev_error_num, dev_acc_num = checkAccuracy(dev_pred_OH, y_dev_inflection_OH)
dev_error_num, dev_acc_num

34/34 [==============================] - 0s 7ms/step


(97, 0.9084905660377358)

In [ ]:
dev_errorAnalysis = reportErrorAnalysis(devData[0], devData[5], dev_pred_OH, y_dev_inflection_OH, mixModel1_path + "/dev_errorAnalysis.csv")
dev_errorAnalysis.head()

,index,lemma,b/s,inflection,predict
0,59,nAdiy,0,>anodiyap,nawAdiy
1,193,>amoriykiy~,1,>amoriykiy~iyn,>amiyrokiy~iyn
2,213,na$AT,0,>ano$iTap,na$ATAt
3,229,mujawohar,1,mujawoharAt,muwahawrrt
4,235,masoEaY,0,masAE,masAEiy


In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(dev_errorAnalysis))

(Broken error %, Sound error %):  (51.546391752577314, 48.45360824742268)


In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(devData[0], dev_pred_OH, dic_lemmaInflection)

(70, 0.9339622641509434)

In [ ]:
# Report Percentage of Unseen Errors in Dev set
errorIndices_dev = dev_errorAnalysis['index'].tolist()
unseenIndices_dev = unseenLemInfIndices_dev_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_dev, unseenIndices_dev))

Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples
(20.618556701030926, 18.51851851851852)


### **Test analyze**

In [ ]:
test_pred = mix_model.predict([X_test_bert, X_test_lemma_OH, X_test_genRat])
test_pred_OH = oneHot3D(test_pred)
test_error_num, test_acc_num = checkAccuracy(test_pred_OH, y_test_inflection_OH)
test_error_num, test_acc_num

33/33 [==============================] - 0s 10ms/step


(87, 0.9153696498054474)

In [ ]:
test_errorAnalysis = reportErrorAnalysis(testData[0], testData[5], test_pred_OH, y_test_inflection_OH, mixModel1_path + "/test_errorAnalysis.csv")

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(test_errorAnalysis))

(Broken error %, Sound error %):  (45.97701149425287, 54.02298850574713)


In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(testData[0], test_pred_OH, dic_lemmaInflection)

(70, 0.9319066147859922)

In [ ]:
# Report Percentage of Unseen Errors in Test set
errorIndices_test = test_errorAnalysis['index'].tolist()
unseenIndices_test = unseenLemInfIndices_test_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_test, unseenIndices_test))

Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples
(17.24137931034483, 17.045454545454543)


## **Mix Model (2 Networks: Classification + Machine Translation)**
with 2 outputs(inflection + BS)

In [ ]:
mixModel1_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Machine Translation Models/Mix Model 2'

In [ ]:
BSClassification_path = '/content/drive/MyDrive/AI Projects/Arabic Broken Plural/Results/Classification Models'
BSClassification_model_path = BSClassification_path + '/model 1/weights.hdf5'
BSClassification_model_structure_path = BSClassification_path + "/model 1/structure.h5"

BSC_model = load_model(BSClassification_model_path)
BSC_model_struct = load_model(BSClassification_model_structure_path)

BSC_model_struct.set_weights(BSC_model.get_weights())
BSC_model_struct.trainable = False
# print(BSC_model.layers[1].get_weights()[0][0])
# print(BSC_model_struct.layers[1].get_weights()[0][0])

In [ ]:
input_lemma = Input(shape=(lemma_length, encode_size), name='Lemma')
input_genRat = Input(shape=(genRat_length, ), name='Gender-Rational')

cls = BSC_model_struct([input_lemma, input_genRat])
lstm1 = Bidirectional(LSTM(256, name='Lstm1'), name='Encoder')(input_lemma)

c = Concatenate(axis=1, name='Concatenate')([lstm1, input_genRat, cls])

d = Dense(200, activation='relu', name='Dense1')(c)
r = RepeatVector(inflection_length, name='RepeatVector')(d)

lstm2 = Bidirectional(LSTM(256, return_sequences=True, name='Lstm2'), name='Decoder')(r)
output = TimeDistributed(Dense(encode_size, activation='softmax', name='Dense2'), name='Inflection')(lstm2)

mix_model = Model(inputs=[input_lemma, input_genRat], outputs=[output,cls])

checkpoint_filepath = mixModel1_path + '/weights.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_Inflection_accuracy', mode='max', save_best_only=True, verbose=0)

mix_model.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
mix_model.layers[3].layers[1].get_weights()[0][0]

In [ ]:
with tf.device('/GPU:0'):
  mix_model.fit([X_train_lemma_OH, X_train_genRat], [y_train_inflection_OH, y_train_class], batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=([X_dev_lemma_OH, X_dev_genRat], [y_dev_inflection_OH,y_dev_class]))  # starts training

In [ ]:
mix_model = load_model(checkpoint_filepath)
mix_model.evaluate([X_test_lemma_OH, X_test_genRat], [y_test_inflection_OH, y_test_class])

In [ ]:
print(mix_model.summary())
plot_model(mix_model, to_file=mixModel1_path+'/architecture.png', show_shapes=True, show_layer_names=True)

### **Train analyze**

In [ ]:
train_pred,bs_pred = mix_model.predict([X_train_lemma_OH, X_train_genRat])
train_pred_OH = oneHot3D(train_pred)
train_error_num, train_acc_num = checkAccuracy(train_pred_OH, y_train_inflection_OH)
train_error_num, train_acc_num

In [ ]:
y_train_inflection_OH.shape

In [ ]:
train_errorAnalysis = reportErrorAnalysis(trainData[0], trainData[5], train_pred_OH, y_train_inflection_OH, mixModel1_path + "/train_errorAnalysis.csv")
train_errorAnalysis

In [ ]:
reportErrorAnalysis_inflectionWithBS(trainData[0], trainData[5],bs_pred, train_pred_OH, y_train_inflection_OH, mixModel1_path + "/train_errorAnalysis_with_bs.csv")

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(train_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(trainData[0],train_pred_OH,dic_lemmaInflection)

### **Dev analyze**

In [ ]:
dev_pred,bs_pred = mix_model.predict([X_dev_lemma_OH, X_dev_genRat])
dev_pred_OH = oneHot3D(dev_pred)
dev_error_num, dev_acc_num = checkAccuracy(dev_pred_OH, y_dev_inflection_OH)
dev_error_num, dev_acc_num

In [ ]:
dev_errorAnalysis = reportErrorAnalysis_inflectionWithBS(devData[0], devData[5],bs_pred, dev_pred_OH, y_dev_inflection_OH, mixModel1_path + "/dev_errorAnalysis_with_bs.csv")
dev_errorAnalysis

In [ ]:
bstuple = [dev_errorAnalysis['b/s'], dev_errorAnalysis['b/s predict']]
c = 0
n = len(bstuple[0])
for i in range(n):
  if not bstuple[0][i] == bstuple[1][i]: c+=1

c/n

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(dev_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(devData[0], dev_pred_OH, dic_lemmaInflection)

In [ ]:
# Report Percentage of Unseen Errors in Dev set
errorIndices_dev = dev_errorAnalysis['index'].tolist()
unseenIndices_dev = unseenLemInfIndices_dev_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_dev, unseenIndices_dev))

### **Test analyze**

In [ ]:
test_pred,bs_pred = mix_model.predict([X_test_lemma_OH, X_test_genRat])
test_pred_OH = oneHot3D(test_pred)
test_error_num, test_acc_num = checkAccuracy(test_pred_OH, y_test_inflection_OH)
test_error_num, test_acc_num

In [ ]:
test_errorAnalysis = reportErrorAnalysis_inflectionWithBS(testData[0], testData[5],bs_pred, test_pred_OH, y_test_inflection_OH, mixModel1_path + "/test_errorAnalysis_with_bs.csv")
test_errorAnalysis

In [ ]:
bstuple = [test_errorAnalysis['b/s'], test_errorAnalysis['b/s predict']]
c = 0
n = len(bstuple[0])
for i in range(n):
  if not bstuple[0][i] == bstuple[1][i]: c+=1

c/n

In [ ]:
print('(Broken error %, Sound error %): ', reportPercentageOfBrokenSoundErrors(test_errorAnalysis))

In [ ]:
checkAccuracy_anyAcceptableInflectionInDataset(testData[0], test_pred_OH, dic_lemmaInflection)

In [ ]:
# Report Percentage of Unseen Errors in Test set
errorIndices_test = test_errorAnalysis['index'].tolist()
unseenIndices_test = unseenLemInfIndices_test_df['index'].tolist()

print('Percentage of Unseen Errors with respect to Total Errors, Percentage of Unseen Errors with respect to Total Unseen Samples')
print(reportPercentageOfUnseenErrors(errorIndices_test, unseenIndices_test))